In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow.keras import layers, models

In [17]:
df = pd.read_csv("D:/credit_card_fraud_dataset.csv")
df = df.select_dtypes(include=[np.number])

In [19]:
X = df.drop("IsFraud", axis=1)
y = df["IsFraud"]

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [23]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [25]:
class WeightedF1(tf.keras.metrics.Metric):
    def __init__(self, name='weighted_f1', **kwargs):
        super(WeightedF1, self).__init__(name=name, **kwargs)
        self.tp = self.add_weight(name='tp', initializer='zeros')
        self.fp = self.add_weight(name='fp', initializer='zeros')
        self.fn = self.add_weight(name='fn', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.cast(tf.greater(y_pred, 0.5), tf.float32)
        y_true = tf.cast(y_true, tf.float32)

        tp = tf.reduce_sum(tf.cast(y_true * y_pred, self.dtype))
        fp = tf.reduce_sum(tf.cast((1 - y_true) * y_pred, self.dtype))
        fn = tf.reduce_sum(tf.cast(y_true * (1 - y_pred), self.dtype))

        self.tp.assign_add(tp)
        self.fp.assign_add(fp)
        self.fn.assign_add(fn)

    def result(self):
        precision = self.tp / (self.tp + self.fp + 1e-7)
        recall = self.tp / (self.tp + self.fn + 1e-7)
        return 2 * (precision * recall) / (precision + recall + 1e-7)

    def reset_states(self):
        self.tp.assign(0)
        self.fp.assign(0)
        self.fn.assign(0)

In [27]:
model = models.Sequential([
    layers.Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', WeightedF1()])


C:\Users\indum\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [29]:
model.fit(X_train, y_train, epochs=5, batch_size=2048, validation_split=0.2, verbose=1)

Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 29ms/step - accuracy: 0.9413 - loss: 0.4964 - weighted_f1: 0.0113 - val_accuracy: 0.9898 - val_loss: 0.2659 - val_weighted_f1: 0.0000e+00
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9895 - loss: 0.2180 - weighted_f1: 0.0000e+00 - val_accuracy: 0.9898 - val_loss: 0.1162 - val_weighted_f1: 0.0000e+00
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9902 - loss: 0.1011 - weighted_f1: 0.0000e+00 - val_accuracy: 0.9898 - val_loss: 0.0761 - val_weighted_f1: 0.0000e+00
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9898 - loss: 0.0727 - weighted_f1: 0.0000e+00 - val_accuracy: 0.9898 - val_loss: 0.0661 - val_weighted_f1: 0.0000e+00
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9898 - loss: 0.0645 - weighted_f1: 0.0000e+00 - val_accuracy: 0.9898 - val_loss: 0.0633 - val_weighted_f1: 0.0000e+00


In [31]:
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step


C:\Users\indum\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [33]:
tp = np.sum((y_test.values == 1) & (y_pred.flatten() == 1))
fp = np.sum((y_test.values == 0) & (y_pred.flatten() == 1))
fn = np.sum((y_test.values == 1) & (y_pred.flatten() == 0))
custom_f1 = 2 * tp / (2 * tp + fp + fn + 1e-7)

In [35]:
print("\n📊 Metric Comparison Table:")
print(f"{'Metric':<20}{'Value':<10}")
print(f"{'-'*30}")
print(f"{'Accuracy':<20}{model.evaluate(X_test, y_test, verbose=0)[1]:.4f}")
print(f"{'Standard F1':<20}{f1:.4f}")
print(f"{'Custom Weighted F1':<20}{custom_f1:.4f}")
print("\n🧠 Custom Metric Insight:")
print("The custom weighted F1 score balances precision and recall, especially valuable in imbalanced datasets like fraud detection.")
print("It helps evaluate performance beyond mere accuracy, which can be misleading when classes are imbalanced.")


📊 Metric Comparison Table:
Metric              Value     
------------------------------
Accuracy            0.9900
Standard F1         0.0000
Custom Weighted F1  0.0000

🧠 Custom Metric Insight:
The custom weighted F1 score balances precision and recall, especially valuable in imbalanced datasets like fraud detection.
It helps evaluate performance beyond mere accuracy, which can be misleading when classes are imbalanced.
